In [32]:
# sklearn 버전확인
import sklearn
print(sklearn.__version__)

0.23.1


In [33]:
# 내장 데이터 로딩
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
model = DecisionTreeClassifier()
train_feature = iris.data
train_label = iris.target
model.fit(train_feature, train_label)

# train 데이터로만 훈련시키고 검증 하는 케이스 => 잘못된 케이스임
pred = model.predict(train_feature)
print(f" 정확도 : {accuracy_score(train_label, pred)}")

 정확도 : 1.0


In [34]:
# cross validation
# 일반적인 K-Fold cv
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
all_acc = []
fold_idx = 0

features = iris.data
labels = iris.target

for train_idx, test_idx in kf.split(features):
    
    train_x, train_y = features[train_idx], labels[train_idx]
    test_x, test_y = features[test_idx], labels[test_idx]
    
    model = DecisionTreeClassifier()
    model.fit(train_x, train_y)
    pred_y = model.predict(test_x)
    acc = accuracy_score(test_y, pred_y)
    
    fold_idx += 1
    
    all_acc.append(acc)

print(f"KFold 모두 수행 후 평균 예측도 : {np.mean(all_acc)}")

KFold 모두 수행 후 평균 예측도 : 0.9266666666666667


In [35]:
# Stratified Cross validation
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)
all_acc = []
fold_idx = 0

features = iris.data
labels = iris.target

for train_idx, test_idx in skf.split(features, labels):
    
    train_x, train_y = features[train_idx], labels[train_idx]
    test_x, test_y = features[test_idx], labels[test_idx]
    
    model = DecisionTreeClassifier()
    model.fit(train_x, train_y)
    pred_y = model.predict(test_x)
    acc = accuracy_score(test_y, pred_y)
    
    fold_idx += 1
    
    all_acc.append(acc)

print(f"KFold 모두 수행 후 평균 예측도 : {np.mean(all_acc)}")

KFold 모두 수행 후 평균 예측도 : 0.9600000000000002


In [36]:
# cross_val_score로 집약해서 교차검증
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
import numpy as np

iris_data = load_iris()
features = iris_data.data
labels = iris.target

model = DecisionTreeClassifier()

scores = cross_val_score(model, features, labels,
                        scoring='accuracy', cv=5)

print(f"한 번씩 검증 때마다 accuracy : {scores}")
print()
print(f"5번 모두 검증한 accuracy 총 평균 : {np.mean(scores)}")

한 번씩 검증 때마다 accuracy : [0.96666667 0.96666667 0.9        0.96666667 1.        ]

5번 모두 검증한 accuracy 총 평균 : 0.9600000000000002


In [40]:
# 최고의 Hyperparameter 찾기 위한 GridsearchCV 사용
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

iris_data = load_iris()
train_x, test_x, train_y, test_y = train_test_split(iris_data.data,
                                                   iris_data.target,
                                                   test_size=0.2,
                                                   random_state=45)
model = DecisionTreeClassifier()
# parameter 넣어줄 값들 dict 형태로 정의해주기
h_para = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

In [41]:
import pandas as pd

grid_dtree = GridSearchCV(model, param_grid=h_para,
                         cv=5, refit=True, return_train_score=True)
# GridSearchCV 인자설명
# cv = 하나의 파라미터 쌍으로 모델링할 때 train, test 교차검증을 3번실시하겠다는 뜻
# refit=True : GridSearch한 후 가장 최고로 좋은 파라미터로 학습시켜 놓겠다.
# ㄴ> 이것 때문에 애초에 GridSearchCV 적용한 객체만으로 최적의 파라미터 적용된 모델로드 가능

# GridSearch 하면서 모든 파라미터값들에 대해 학습 수행
grid_dtree.fit(train_x, train_y)

# 각 파라미터값들에 대한 모델 결과값들이 cv_results_ 객체에 할당됨
scores_df = pd.DataFrame(grid_dtree.cv_results_)

# score 결과값(ndarray형태로 할당됨) 중 특정 칼럼들만 가져오기 
scores_df[['params', 'mean_test_score', 'rank_test_score', 
           'split0_test_score', 'split1_test_score', 'split2_test_score']]


,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.683333,5,0.666667,0.708333,0.708333
1,"{'max_depth': 1, 'min_samples_split': 3}",0.683333,5,0.666667,0.708333,0.708333
2,"{'max_depth': 2, 'min_samples_split': 2}",0.966667,1,0.958333,1.000000,0.916667
3,"{'max_depth': 2, 'min_samples_split': 3}",0.966667,1,0.958333,1.000000,0.916667
4,"{'max_depth': 3, 'min_samples_split': 2}",0.950000,3,0.958333,0.958333,0.916667
5,"{'max_depth': 3, 'min_samples_split': 3}",0.941667,4,0.958333,0.958333,0.916667


In [42]:
# 최적의 파라미터는 best_params_에 할당되어있음
print(f"최적의 파라미터 : {grid_dtree.best_params_}")
print(f"최적의 파라미터로 모델의 정확도 : {grid_dtree.best_score_}")

print()
print()

# 최적의 파라미터로 학습되어 있는 모델링 할당
estimator = grid_dtree.best_estimator_

# 최적의 모델로 예측해보고 실제값이랑 정확도 비교
pred_y = estimator.predict(test_x)
print(f"실제값과 예측값 정확도 : {accuracy_score(test_y, pred_y)}")

최적의 파라미터 : {'max_depth': 2, 'min_samples_split': 2}
최적의 파라미터로 모델의 정확도 : 0.9666666666666668


실제값과 예측값 정확도 : 0.9333333333333333
